In [93]:
import random
import uuid

class Person(object):

    def __init__(self, uid, s_type):
        self._uid = uid
        self._s_type = s_type

    def get_uid(self):
        return self._uid
    
    def get_s_type(self):
        return self._s_type


class FriendNetwork(object):

    def __init__(self, people_num, connections_num):
        self._people_num = people_num
        self._connections_num = connections_num
        self._graph = self._generate_graph()

    def _generate_graph(self):

        people = []
        for person_index in range(self._people_num):
            uid = str(uuid.uuid4())
            s_type = 'female' if person_index < (self._people_num // 2)  else 'male'
            people.append(Person(uid, s_type))

        conn_num = 0
        graph = {}
        graph_aux = {} # criando um grafo auxiliar para agilizar algumas buscas
        while conn_num < self._connections_num:
            person, friend = random.sample(people, 2)
            person_uid = person.get_uid()
            friend_uid = friend.get_uid()

            if person_uid not in graph:
                graph[person_uid] = {
                    'this': person,
                    'friends': []
                }
                # criando um índice auxiliar para os vizinhos de cada vértice inserido no grafo
                graph_aux[person_uid] = {}

            if friend_uid not in graph:
                graph[friend_uid] = {
                    'this': friend,
                    'friends': []
                }
                # criando um índice auxiliar para os vizinhos de cada vértice inserido no grafo
                graph_aux[friend_uid] = {} 

            # if person_uid == friend_uid or \
            #     friend in graph[person_uid]['friends']: # fazer essa verificação em um índice auxiliar
            #     continue
            if person_uid == friend_uid or \
                friend_uid in graph_aux[person_uid]: # fazer essa verificação em um índice auxiliar
                continue

            graph[person_uid]['friends'].append(friend)
            graph[friend_uid]['friends'].append(person)
            # adicionar vizinho também nos índices do grafo auxiliar
            graph_aux[person_uid][friend_uid] = True
            graph_aux[friend_uid][person_uid] = True
            conn_num += 1

        people_to_remove = []
        for person_uid in graph:
            friends_types = [*map(lambda p: p.get_s_type(), graph[person_uid]['friends'])]
            person_type = graph[person_uid]['this'].get_s_type()
            if ('male' not in friends_types or 'female' not in friends_types) and person_type in friends_types:
                people_to_remove.append({'person_uid': person_uid, 'remove_from': graph[person_uid]['friends']})

        for person_props in people_to_remove:
            for friend in person_props['remove_from']:
                person_index = [*map(lambda friend: friend.get_uid(),
                    graph[friend.get_uid()]['friends'])].index(person_props['person_uid'])
                del graph[friend.get_uid()]['friends'][person_index]
            del graph[person_props['person_uid']]

        return graph
    
    def get_person_by_uid(self, uid):
        return self._graph[uid]['this']
    
    def _search(self, person_uid, friend_uid):
        '''
        person_uid: start node
        friend_uid: target node
        '''
        
        visited = {person_uid}
        queue = [person_uid]
        parent = {
            person_uid: None
        }
        found = False
        
        while queue:
            
            current = queue.pop(0)
            
            if current == friend_uid:
                found = True
                break
            
            for friend in self._graph[person_uid]["friends"]:
                next_node = friend._uid
                if next_node not in visited:
                    visited.add(next_node)
                    queue.append(next_node)
                    parent[next_node] = current
        
        return visited 

    def get_separation_degree(self):
       
        total_paths_len = 0
        for _ in range(100):
            person_uid, friend_uid = random.sample([*self._graph.keys()], 2)
            path = self._search(person_uid, friend_uid)
            total_paths_len += len(path) - 1

        return total_paths_len / 100




friend_network = FriendNetwork(100, 2000)

In [94]:
friend_network.get_separation_degree()

{'3f652151-a75f-4579-92b9-2c8f4cdca8a0', 'ed966deb-fa9c-493b-86a9-8a856d048561', '717842eb-a387-4df5-a178-7b8457b95c62', '7ca34e7c-b20f-43d2-b4eb-53ce1acd9ea0', 'e75e015c-7c20-4a5d-91de-2a166b22c08e', '2b061d89-05c2-4069-b2c0-f99b92f0ff7b', 'fdb87155-38d4-464a-aec9-0c722e1ab452', 'd7264d19-47ec-4eb6-ab32-f1dc957b03c7', 'bb0bc281-31d1-4790-815c-314a208355a9', '42da5fec-9b94-4024-b10d-5ab630bf9085', 'e720b522-86e8-48a8-bfd7-a46d69330531', '7ac79494-0e07-473c-8417-5d25b521ac0a', 'beacbd87-5c90-4b14-bfae-7381674287ff', '63e04beb-7763-40c8-a75a-6561fe8e1eab', '6ce354b5-7707-409a-bd05-6decf87095df', '019d1601-14cb-4e57-905b-485d8188de51', '25835db3-7769-4771-b7f1-cde5fa6f6cae', '79652ed2-89d2-4247-90a9-507422164b6d', '400f6bb9-24c2-4de8-80f0-84c62a46e76c', '4f92bb1c-93e9-49df-9cb1-d8273005962f', 'c68f21fb-a1d2-4acb-9142-ae9c9652ab6c', 'bfc83f1f-36ab-4902-8a43-bb47327c5324', '1e8ee0b3-ea8c-4171-a70c-7ad6e58cadc1', 'a8fe29ae-f42d-4e1a-9744-f4e498396587', '3ff4b43c-b53e-4560-b628-b0e91d85a992',

39.74

In [58]:
parents = [0,1,2,3,4,5,6,7,8,9,10,11]

while parents is not None:
    parents.pop(0)
    print(parents)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[3, 4, 5, 6, 7, 8, 9, 10, 11]
[4, 5, 6, 7, 8, 9, 10, 11]
[5, 6, 7, 8, 9, 10, 11]
[6, 7, 8, 9, 10, 11]
[7, 8, 9, 10, 11]
[8, 9, 10, 11]
[9, 10, 11]
[10, 11]
[11]
[]


IndexError: pop from empty list

In [96]:
p_uid, f_uid = random.sample([*friend_network._graph.keys()], 2)

In [99]:
friend_network._graph[p_uid]["this"].__dict__

{'_uid': 'a01bcacd-c656-4bed-904a-4c0b8f28be7b', '_s_type': 'female'}

In [101]:
from enum import Enum

class Types(str, Enum):
    MALE = "male"
    FEMALE = "female"

In [106]:
assert friend_network._graph[p_uid]["this"]._s_type == Types.MALE

AssertionError: 

In [107]:
assert friend_network._graph[p_uid]["this"]._s_type == Types.FEMALE

In [109]:
my_list = [0,1,2]

In [110]:
my_list.append(3)

my_list

[0, 1, 2, 3]